# Tratando Dataset

In [3]:
!curl -L -o /tmp/tcc_alpd/archive.tar.gz "https://www.dropbox.com/scl/fi/khnnv0x8pcpbzlgfevqac/archive.tar.gz?rlkey=55w3pkxp02szjpqw22df7hmn8&dl=0"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256    0   256    0     0    214      0 --:--:--  0:00:01 --:--:--   214
100   491    0   491    0     0    206      0 --:--:--  0:00:02 --:--:--     0
100  202M  100  202M    0     0  10.8M      0  0:00:18  0:00:18 --:--:-- 7732k


In [4]:
!tar xfv /tmp/tcc_alpd/archive.tar.gz -C /tmp/tcc_alpd

archive/
archive/.DS_Store
archive/images/
archive/annotations/
archive/annotations/Cars213.xml
archive/annotations/Cars207.xml
archive/annotations/Cars398.xml
archive/annotations/Cars367.xml
archive/annotations/Cars401.xml
archive/annotations/Cars415.xml
archive/annotations/Cars373.xml
archive/annotations/Cars429.xml
archive/annotations/Cars47.xml
archive/annotations/Cars53.xml
archive/annotations/Cars165.xml
archive/annotations/Cars171.xml
archive/annotations/Cars159.xml
archive/annotations/Cars84.xml
archive/annotations/Cars90.xml
archive/annotations/Cars91.xml
archive/annotations/Cars85.xml
archive/annotations/Cars158.xml
archive/annotations/Cars170.xml
archive/annotations/Cars164.xml
archive/annotations/Cars52.xml
archive/annotations/Cars46.xml
archive/annotations/Cars428.xml
archive/annotations/Cars414.xml
archive/annotations/Cars372.xml
archive/annotations/Cars366.xml
archive/annotations/Cars400.xml
archive/annotations/Cars399.xml
archive/annotations/Cars206.xml
archive/annotati

In [10]:
import xml.etree.ElementTree as xet
from glob import glob
import pandas as pd
import os

path = glob('/tmp/tcc_alpd/archive/annotations/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)
    

In [8]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/tmp/tcc_alpd/archive/annotations/Cars400.xml,303,340,191,217
1,/tmp/tcc_alpd/archive/annotations/Cars104.xml,195,244,266,282
2,/tmp/tcc_alpd/archive/annotations/Cars224.xml,181,243,250,275
3,/tmp/tcc_alpd/archive/annotations/Cars311.xml,295,355,166,185
4,/tmp/tcc_alpd/archive/annotations/Cars319.xml,167,232,228,259


In [12]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/tmp/tcc_alpd/archive/images',filename_image)
    return filepath_image

In [14]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['/tmp/tcc_alpd/archive/images/Cars400.png',
 '/tmp/tcc_alpd/archive/images/Cars104.png',
 '/tmp/tcc_alpd/archive/images/Cars224.png',
 '/tmp/tcc_alpd/archive/images/Cars311.png',
 '/tmp/tcc_alpd/archive/images/Cars319.png',
 '/tmp/tcc_alpd/archive/images/Cars73.png',
 '/tmp/tcc_alpd/archive/images/Cars11.png',
 '/tmp/tcc_alpd/archive/images/Cars370.png',
 '/tmp/tcc_alpd/archive/images/Cars326.png',
 '/tmp/tcc_alpd/archive/images/Cars304.png',
 '/tmp/tcc_alpd/archive/images/Cars347.png',
 '/tmp/tcc_alpd/archive/images/Cars367.png',
 '/tmp/tcc_alpd/archive/images/Cars186.png',
 '/tmp/tcc_alpd/archive/images/Cars364.png',
 '/tmp/tcc_alpd/archive/images/Cars299.png',
 '/tmp/tcc_alpd/archive/images/Cars351.png',
 '/tmp/tcc_alpd/archive/images/Cars228.png',
 '/tmp/tcc_alpd/archive/images/Cars346.png',
 '/tmp/tcc_alpd/archive/images/Cars278.png',
 '/tmp/tcc_alpd/archive/images/Cars365.png',
 '/tmp/tcc_alpd/archive/images/Cars129.png',
 '/tmp/tcc_alpd/archive/images/Cars292.png',
 '/tmp/tcc_a

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-04-03 18:09:26.728281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 18:09:26.766772: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 18:09:26.767500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 18:09:27.393364: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

2024-04-03 18:09:30.853268: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-03 18:09:32.299282: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


219055592/219055592 [==============================] - 11s 0us/step
